In [173]:
!pip install selenium

In [45]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import bs4
import requests
import time

In [125]:
#เปิดเว็บ Sanook
driver = webdriver.Chrome()
driver.get('https://www.sanook.com/news/')

In [126]:
# ค้นหาช่องค้นหา
search = driver.find_element("xpath",'/html/body/div[2]/div[1]/header/div/div/div[2]/div/form/label/input')

In [127]:
# พิมพ์คำว่า การศึกษาไทย
search.send_keys('การศึกษาไทย')

In [128]:
# กดค้นหา
search.send_keys(Keys.ENTER)

In [129]:
driver.execute_script("document.body.style.zoom='100%'")

# Scraping

In [130]:
# INFINITE scrolling pages
# เลื่อนลงเพื่อให้โหลดข่าวเยอะๆ

previous_height = driver.execute_script('return document.body.scrollHeight')

for i in range(2):
    while True:
        driver.execute_script('window.scrollTo(0,  document.body.scrollHeight);')

        time.sleep(3)

        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == previous_height:
            break
        previous_height = new_height
        

In [131]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [132]:
# เก็บลิ้งค์ข่าวทั้งหมดในหน้า 
links = []
for link_news in soup.findAll('span',{'class':'jsx-2430232205 jsx-2715185928 text-color-news'}):
    links.append(link_news.a['href'])
print(len(links))

152


In [133]:
all_links = []
Title = []
Release_date = [] 
Contents = []

order = 1

a = 1
# เข้าไปทีละข่าว
for link in links:
    
    print(link)
    #time.sleep(3)
    page = requests.get(link , allow_redirects=False)
    content_news = bs4.BeautifulSoup(page.content , "lxml")
    a += 1
    #content_news = bs4.BeautifulSoup(page.content , "lxml")
    
    for e in content_news('body') :
        # หาชื่อข่าว
        title=e.find('div', attrs={'class':'jsx-223461968 EntryHeading clearfix'}).find('h1' , {'class' : 'jsx-223461968 title'}).text
        # หาวันที่เผยแพร่
        release_date=e.find('span',attrs={'class':'jsx-1665670616 infoItem'}).time['datetime']
        # หาเนื้อข่าวทั้งหมด
        #Contents = e.find('div',attrs={'id':'EntryReader_0'}).get_text(strip=True)
        Contents.append(e.find('div',attrs={'id':'EntryReader_0'}).get_text(strip=True))
        
        # เพิ่มเข้าไปใน list
        all_links.append(link)
        Title.append(title)
        Release_date.append(release_date)
        print(f'{order} {link} : {title}')
        print(f'Date : {release_date}')
        
        order += 1

https://www.sanook.com/news/8650938/
1 https://www.sanook.com/news/8650938/ : “ภาวะฉุกเฉินทางการเรียนรู้” เมื่อการศึกษาไทยแขวนอยู่บนเส้นด้าย
Date : 2022-11-07 18:21
https://www.sanook.com/news/8637914/
2 https://www.sanook.com/news/8637914/ : “กระทรวงศึกษาธิการ” ปลดล็อกการศึกษาไทย ด้วยนโยบาย “ซ่อม สร้าง ป้องกัน”
Date : 2022-10-11 17:31
https://www.sanook.com/news/8614190/
3 https://www.sanook.com/news/8614190/ : “ภาวะการเรียนรู้ถดถอย” ภัยเงียบที่กำลังกัดเซาะสังคม
Date : 2022-08-26 17:50
https://www.sanook.com/news/8602854/
4 https://www.sanook.com/news/8602854/ : ครูจุ๊ย กุลธิดา: เปลือยการศึกษาไทยช่วงวิกฤติโควิด-19
Date : 2022-08-04 15:44
https://www.sanook.com/news/8588214/
5 https://www.sanook.com/news/8588214/ : 2 เยาวชนไทยในโอลิมปิกปรัชญาและเหรียญทองแดงแรกในประวัติศาสตร์
Date : 2022-07-06 16:20
https://www.sanook.com/news/8569186/
6 https://www.sanook.com/news/8569186/ : นพ.ยงยุทธ วงศ์ภิรมย์ศานติ์: จิตวิทยาว่าด้วย “ความงมงาย” ของคนไทย
Date : 2022-05-30 13:10
https://www.sanook.com/

In [134]:
import pandas as pd

In [135]:
df = {'Title': Title,
        'Release_date':Release_date,
        'Content':Contents,
        }
# Create Data Frame
df = pd.DataFrame(df)
df

,Title,Release_date,Content
0,“ภาวะฉุกเฉินทางการเรียนรู้” เมื่อการศึกษาไทยแข...,2022-11-07 18:21,Highlightรายงานฉบับพิเศษ “ห้องเรียนฟื้นฟูหลังโ...
1,“กระทรวงศึกษาธิการ” ปลดล็อกการศึกษาไทย ด้วยนโย...,2022-10-11 17:31,การศึกษาของเด็กไทยคือหัวใจสำคัญในการขับเคลื่อน...
2,“ภาวะการเรียนรู้ถดถอย” ภัยเงียบที่กำลังกัดเซาะ...,2022-08-26 17:50,Highlightบทเรียนงานวิจัยเกี่ยวกับพัฒนาการเด็กป...
3,ครูจุ๊ย กุลธิดา: เปลือยการศึกษาไทยช่วงวิกฤติโค...,2022-08-04 15:44,Highlightระบบราชการ ระบบคิดของคนต่อการศึกษา แล...
4,2 เยาวชนไทยในโอลิมปิกปรัชญาและเหรียญทองแดงแรกใ...,2022-07-06 16:20,ชวนรู้จักกับ 2 เยาวชนไทยที่คว้าเหรียญทองแดงและ...
...,...,...,...
147,พล.อ.สุรยุทธ์เผย ในหลวงทรงห่วงใยการศึกษาไทย,2013-07-25 13:03,องคมนตรี สุรยุทธ์ เผย พระบาทสมเด็จพระเจ้าอยู่ห...
148,ฮือฮา! เด็ก 2 ขวบ อัจฉริยะภาษา (ชมคลิป),2013-07-13 13:53,วันที่ 11 ก.ค. ผู้สื่อข่าวรับแจ้งมีเด็กอัจริยะ...
149,ดุสิตโพลคนฝากรมต.ใหม่เร่งแก้ใต้กระตุ้นศก.,2013-07-05 11:23,ดุสิตโพล เผย ประชาชนอยากฝากรัฐมนตรีใหม่ เร่งแก...
150,Sanook social news วันที่ 5 กรกฎาคม 2556,2013-07-05 10:15,คัดเรื่องเด่นประเด็นฮอตในรอบสัปดาห์กับSanook! ...


In [108]:
# Change Data type DATE to datetime
from datetime import datetime
df['Release_date'] = pd.to_datetime(df['Release_date'])

In [109]:
# Check DataType
df.dtypes

Title                   object
Release_date    datetime64[ns]
Content                 object
dtype: object

In [113]:
# ลบพวกตัวแปลกๆ \xa0 , \u200b
df['Content']= df['Content'].replace(to_replace='\xa0', regex=True, value='')
df['Content']= df['Content'].replace(to_replace='\u200b', regex=True, value='')

In [114]:
df['Content'][0]

'เครือซีพีสนับสนุนภาคการศึกษาไทย มอบทุนการศึกษาต่อเนื่อง 41 ปีช่วยเยาวชนขาดแคลนทุนทรัพย์ หวังลดความเหลื่อมล้ำทางสังคม ปีนี้เดินหน้ามอบแก่นิสิตนักศึกษาทั่วประเทศ 262 ทุน ๆ ละ 30,000 บาทพร้อมอัดฉีดเรียนฟรี E Learning ตลอดช่วงรับทุน จากโรงเรียนสอนภาษาระดับโลกเสริมทักษะภาษาอังกฤษให้แข็งแกร่งรองรับเป็นศูนย์กลางอาเซียน ทั้งยังประกาศเพิ่มทุนการศึกษาด้านนวัตกรรมแก่นักเรียนม.ปลายและสายอาชีพเพื่อตอบโจทย์ไทยแลนด์ 4.0 สร้างคนเพื่อขับเคลื่อนประเทศด้วยนวัตกรรม31 พฤษภาคม 2562 – เครือเจริญโภคภัณฑ์ (ซีพี) จัดพิธีมอบทุนการศึกษาประจำปี 2562 แก่นิสิตนักศึกษาทั่วประเทศต่อเนื่องเป็นปีที่ 41 ถือเป็นทุนการศึกษาของภาคเอกชนที่มีประวัติศาสตร์ยาวนานที่สุดทุนหนึ่งของประเทศไทยโดยมีศาสตราจารย์เกียรติคุณ นพ.เกษม วัฒนชัยประธานคณะกรรมการบริหารทุนการศึกษา เครือเจริญโภคภัณฑ์ เป็นประธานมอบทุนการศึกษาฯ พร้อมด้วยนายแพทย์ธีระเกียรติ เจริญเศรษฐศิลป์รองประธานคณะกรรมการบริหารทุนการศึกษา เครือฯดร.คุณหญิง กษมา วรวรรณ ณ อยุธยากรรมการบริหารทุนการศึกษา เครือฯนายศุภชัย เจียรวนนท์ประธานคณะผู้บริหาร เครือเจริญโภคภัณฑ์ และคณะกรรมการบริห

### Save to CSV

In [115]:
df.to_csv('News_Sanook_5.csv', index=False, encoding ='utf-8-sig')

### or Save to json

In [116]:
df['Release_date'] = (df['Release_date']).astype(str)

In [117]:
df = df.to_dict('records')

In [118]:
import json
with open('News_Sanook_5.json', 'w', encoding ='utf8') as json_file:
    json.dump(df, json_file)